In [8]:
! pip install keras_tuner

## Install libraries

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
print(tf.__version__)

2.14.0


## Data Collection

In [11]:
fashion_mnist = keras.datasets.fashion_mnist

In [12]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

## Data Normalization

In [13]:
X_train = X_train/255.0
X_test = X_test/ 255.0

In [14]:
# Reshape the image data to 4 dimension
X_train = X_train.reshape(len(X_train), 28, 28, 1)
X_test = X_test.reshape(len(X_test), 28, 28, 1)

In [22]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
          kernel_size=hp.Choice('cov_2_kernel', values = [3,5]),
          activation = 'relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
          kernel_size=hp.Choice('cov_2_kernel', values = [3,5]),
          activation = 'relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units= hp.Int('dense_1_unit', min_value = 32, max_value = 128, step = 16),
          activation = 'relu'
      ),
      keras.layers.Dense(10, activation='softmax')

      ])

  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )

  return model

In [23]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [24]:
tunner_search = RandomSearch(build_model,
                             objective='val_accuracy',
                             max_trials=5,
                             directory = 'output',
                             project_name = 'Mnist Fashion')

Reloading Tuner from output/Mnist Fashion/tuner0.json


In [25]:
tunner_search.search(X_train, Y_train, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 29m 15s]
val_accuracy: 0.9111666679382324

Best val_accuracy So Far: 0.9111666679382324
Total elapsed time: 01h 44m 29s


In [26]:
model = tunner_search.get_best_models(num_models=1)[0]

In [30]:
from ast import Num
tunner_search.get_best_hyperparameters()

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 96)        83040     
                                                                 
 flatten (Flatten)           (None, 55296)             0         
                                                                 
 dense (Dense)               (None, 128)               7078016   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 7163306 (27.33 MB)
Trainable params: 7163306 (27.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# train using the best model
model.fit(X_train, Y_train, epochs = 10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 566s 335ms/step - loss: 0.1612 - accuracy: 0.9401 - val_loss: 0.2459 - val_accuracy: 0.9145
Epoch 5/10
1688/1688 [==============================] - 553s 327ms/step - loss: 0.1098 - accuracy: 0.9586 - val_loss: 0.2697 - val_accuracy: 0.9225
Epoch 6/10
1688/1688 [==============================] - 551s 327ms/step - loss: 0.0726 - accuracy: 0.9742 - val_loss: 0.2992 - val_accuracy: 0.9185
Epoch 7/10
1688/1688 [==============================] - 567s 336ms/step - loss: 0.0500 - accuracy: 0.9824 - val_loss: 0.3588 - val_accuracy: 0.9125
Epoch 8/10
1688/1688 [==============================] - 561s 332ms/step - loss: 0.0342 - accuracy: 0.9874 - val_loss: 0.3939 - val_accuracy: 0.9097
Epoch 9/10
1688/1688 [==============================] - 554s 328ms/step - loss: 0.0266 - accuracy: 0.9903 - val_loss: 0.4785 - val_accuracy: 0.9130
Epoch 10/10
1688/1688 [==============================] - 547s 324ms/step - loss: 0.0222 - accuracy: 0.9922 - val

In [32]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 27s 88ms/step - loss: 0.5204 - accuracy: 0.9083


[0.5203821659088135, 0.90829998254776]